# Quickstart

In this quickstart you will create a simple LLM Chain and learn how to log it and get feedback on an LLM response.

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.parent.resolve()))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""
None

## Setup
### Add API keys
For this quickstart you will need Open AI and Huggingface keys

In [2]:
from trulens_eval.keys import setup_keys

setup_keys(
    OPENAI_API_KEY="to fill in",
    HUGGINGFACE_API_KEY="to fill in"
)

✅ Key OPENAI_API_KEY set from environment.
✅ Key HUGGINGFACE_API_KEY set from environment.


### Import from LangChain and TruLens

In [3]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import TruChain, Feedback, Huggingface, Tru
tru = Tru()

# Imports from langchain to build app. You may need to install langchain first
# with the following:
# ! pip install langchain>=0.0.170
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

### Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [4]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

### Send your first request

In [5]:
prompt_input = '¿que hora es?'

In [6]:
llm_response = chain(prompt_input)

display(llm_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: ¿que hora es?

> Finished chain.


{'prompt': '¿que hora es?',
 'text': '\n\nLa hora actual en tu zona es la hora específica que indica el reloj. Dependiendo de en qué parte del mundo estés, la hora varía. Por ejemplo, la diferencia de horario entre la costa este de Estados Unidos y la costa oeste de Estados Unidos es de tres horas.'}

## Initialize Feedback Function(s)

In [7]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on_input_output()
# By default this will check language match on the main app input and main app
# output.

✅ In language_match, input text1 will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In language_match, input text2 will be set to *.__record__.main_output or `Select.RecordOutput` .


## Instrument chain for logging with TruLens

In [8]:
truchain = TruChain(chain,
    app_id='Chain3_ChatApplication',
    feedbacks=[f_lang_match])

✅ app Chain3_ChatApplication -> default.sqlite
✅ feedback def. feedback_definition_hash_1e615cacc806b2003d759e0487e62acf -> default.sqlite


In [14]:
# Instrumented chain can operate like the original:
llm_response = truchain(prompt_input)

display(llm_response)



> Entering new LLMChain chain...

> Finished chain.


{'prompt': '¿que hora es?',
 'text': '\n\nLa hora actual en la mayoría de países es la hora de Greenwich, o UTC (Coordinated Universal Time). Esto significa que, en la mayoría de lugares, la hora es UTC + 0.'}

✅ record record_hash_c73e0a479bc69bfd2456745713b9bff5 from Chain3_ChatApplication -> default.sqlite
✅ feedback feedback_result_hash_303d5644b52403f29744949a1689e223 on record_hash_c73e0a479bc69bfd2456745713b9bff5 -> default.sqlite


## Explore in a Dashboard

In [13]:
tru.run_dashboard(
    force=True,
    _dev=Path.cwd().parent.parent.parent.resolve()
) # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.1.10.101:8502 .


### Chain Leaderboard

Understand how your LLM application is performing at a glance. Once you've set up logging and evaluation in your application, you can view key performance statistics including cost and average feedback value across all of your LLM apps using the chain leaderboard. As you iterate new versions of your LLM application, you can compare their performance across all of the different quality metrics you've set up.

Note: Average feedback values are returned and displayed in a range from 0 (worst) to 1 (best).

![Chain Leaderboard](https://www.trulens.org/Assets/image/Leaderboard.png)

To dive deeper on a particular chain, click "Select Chain".

### Understand chain performance with Evaluations
 
To learn more about the performance of a particular chain or LLM model, we can select it to view its evaluations at the record level. LLM quality is assessed through the use of feedback functions. Feedback functions are extensible methods for determining the quality of LLM responses and can be applied to any downstream LLM task. Out of the box we provide a number of feedback functions for assessing model agreement, sentiment, relevance and more.

The evaluations tab provides record-level metadata and feedback on the quality of your LLM application.

![Evaluations](https://www.trulens.org/Assets/image/Leaderboard.png)

### Deep dive into full chain metadata

Click on a record to dive deep into all of the details of your chain stack and underlying LLM, captured by tru_chain.

![Explore a Chain](https://www.trulens.org/Assets/image/Chain_Explore.png)

If you prefer the raw format, you can quickly get it using the "Display full chain json" or "Display full record json" buttons at the bottom of the page.

Note: Feedback functions evaluated in the deferred manner can be seen in the "Progress" page of the TruLens dashboard.

## Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all